#installs

In [ ]:
# ! pip install -r requirements.txt

In [ ]:
# ! pip install fastapi kaleido python-multipart uvicorn numpy==1.24.1

In [ ]:
!pip install torchmetrics[image]
!pip install torch-fidelity

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.4/840.4 kB 9.9 MB/s eta 0:00:00


#imports

In [ ]:
import torch
import albumentations as alb
from albumentations.pytorch import ToTensorV2
import torch.nn as nn
import pandas as pd
from PIL import Image
import os
from torch.utils.data import Dataset
import numpy as np
import sys
from torch.utils.data import DataLoader
import torch.optim as optim
from tqdm import tqdm
from torchvision.utils import save_image
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision import datasets, transforms as tf
from torchvision.models import inception_v3
import albumentations as alb

In [ ]:
from dataset import *
from utils import *
from model import *
from evaluation import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Configuration

In [ ]:


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
TRAIN_DIR = "/content/drive/MyDrive/Colab Notebooks/Steganography/data/train"
VAL_DIR = "/content/drive/MyDrive/Colab Notebooks/Steganography/data/test"
BATCH_SIZE = 8
LEARNING_RATE = 1e-5
LAMBDA_IDENTITY = 0.0
LAMBDA_CYCLE = 10
NUM_WORKERS = 2
NUM_EPOCHS = 1
LOAD_MODEL = False
SAVE_MODEL = True
CHECKPOINT_GEN_X= "genX.pth.tar"
CHECKPOINT_GEN_Y = "genY.pth.tar"
CHECKPOINT_DISC_X = "discX.pth.tar"
CHECKPOINT_DISC_Y = "discY.pth.tar"
SAVE_PATH = '/content/drive/MyDrive/Colab Notebooks/Steganography/models'

### Transformations before feeding
# transforms = alb.Compose(
#     [
#         alb.Resize(width=256, height=256),
#         alb.HorizontalFlip(p=0.5),
#         alb.Normalize(mean=[0.5, 0.5, 0.5], std=[
#                       0.5, 0.5, 0.5], max_pixel_value=255),
#         ToTensorV2(),
#     ],
#     additional_targets={"image0": "image"},
# )

# Training

In [ ]:
transforms_alb = alb.Compose(
    [
        alb.Resize(width=256, height=256),
        ToTensorV2(),
    ],
    additional_targets={"image0": "image"},
)

In [ ]:
def train_fn(disc_X, disc_Y, gen_Y, gen_X, loader, opt_disc, opt_gen, l1, mse, d_scaler, g_scaler, embed, steganolysis):
    X_reals = 0
    X_fakes = 0
    loop = tqdm(loader, leave=True)

    for idx, (Y, X) in enumerate(loop):
        Y = Y.to(DEVICE).type(torch.float16)
        X = X.to(DEVICE).type(torch.float16)

        # Train Discriminators X and Y
        with torch.cuda.amp.autocast():
            fake_X = gen_X(Y)

            D_X_real = disc_X(X)
            D_X_fake = disc_X(fake_X.detach())
            X_reals += D_X_real.mean().item()
            X_fakes += D_X_fake.mean().item()
            D_X_real_loss = mse(D_X_real, torch.ones_like(D_X_real))
            D_X_fake_loss = mse(D_X_fake, torch.zeros_like(D_X_fake))
            D_X_loss = D_X_real_loss + D_X_fake_loss

            # steganolysis
            embed_X = embed(fake_X.detach().type(torch.uint8), text='SECRET')
            STEGO_LOSS_X = mse(steganolysis(fake_X.detach().type(torch.uint8)), steganolysis(embed_X.detach().type(torch.uint8)))

            fake_Y = gen_Y(X)

            D_Y_real = disc_Y(Y)
            D_Y_fake = disc_Y(fake_Y.detach())
            D_Y_real_loss = mse(D_Y_real, torch.ones_like(D_Y_real))
            D_Y_fake_loss = mse(D_Y_fake, torch.zeros_like(D_Y_fake))
            D_Y_loss = D_Y_real_loss + D_Y_fake_loss

            # steganolysis
            embed_Y = embed(fake_Y.detach().type(torch.uint8), text='SECRET')
            STEGO_LOSS_Y = mse(steganolysis(fake_Y.detach().type(torch.uint8)), steganolysis(embed_Y.detach().type(torch.uint8)))




            # put it togethor
            D_loss = (D_X_loss + D_Y_loss + STEGO_LOSS_X + STEGO_LOSS_Y) / 4


        opt_disc.zero_grad()
        d_scaler.scale(D_loss).backward()
        d_scaler.step(opt_disc)
        d_scaler.update()



        # Train Generators H and Z
        with torch.cuda.amp.autocast():
            # adversarial loss for both generators
            D_X_fake = disc_X(fake_X)
            D_Y_fake = disc_Y(fake_Y)
            loss_G_X = mse(D_X_fake, torch.ones_like(D_X_fake))
            loss_G_Y = mse(D_Y_fake, torch.ones_like(D_Y_fake))

            # cycle loss
            cycle_X = gen_Y(fake_X)
            cycle_Y = gen_X(fake_Y)
            cycle_Y_loss = l1(X, cycle_Y)
            cycle_X_loss = l1(Y, cycle_X)

            # identity loss (remove these for efficiency if you set lambda_identity=0)
            identity_Y = gen_Y(Y)
            identity_X = gen_X(X)
            identity_Y_loss = l1(Y, identity_Y)
            identity_X_loss = l1(X, identity_X)

            # add all togethor
            G_loss = (
                loss_G_Y
                + loss_G_X
                + cycle_Y_loss * LAMBDA_CYCLE
                + cycle_X_loss * LAMBDA_CYCLE
                + identity_X_loss * LAMBDA_IDENTITY
                + identity_Y_loss * LAMBDA_IDENTITY
            )

        opt_gen.zero_grad()
        g_scaler.scale(G_loss).backward()
        g_scaler.step(opt_gen)
        g_scaler.update()



        if idx % 5 == 0:
            save_image(X * 0.5 + 0.5, f"/content/drive/MyDrive/Colab Notebooks/Steganography/outputs_ViT/X_ORG_{idx}.png")
            save_image(Y * 0.5 + 0.5, f"/content/drive/MyDrive/Colab Notebooks/Steganography/outputs_ViT/Y_ORG_{idx}.png")
            save_image(fake_X * 0.5 + 0.5, f"/content/drive/MyDrive/Colab Notebooks/Steganography/outputs_ViT/X_{idx}.png")
            save_image(fake_Y * 0.5 + 0.5, f"/content/drive/MyDrive/Colab Notebooks/Steganography/outputs_ViT/Y_{idx}.png")


        loop.set_postfix(
            X_real=X_reals / (idx + 1),X_fake=X_fakes / (idx + 1),
            Generator_Loss=G_loss.item(), Discriminator_Loss=D_loss.item()
            )

    return G_loss.item(), D_loss.item()

In [ ]:
def train():
    global metrics
    # disc_X = Discriminator(in_channels=3).to(DEVICE)
    disc_X = ViT((3, 256, 256), n_patches=16, n_blocks=2, hidden_d=8, n_heads=2, out_d=1)
    # disc_Y = Discriminator(in_channels=3).to(DEVICE)
    disc_Y = ViT((3, 256, 256), n_patches=16, n_blocks=2, hidden_d=8, n_heads=2, out_d=1)
    gen_Y = UNET(in_channels=3).to(DEVICE)
    gen_X = UNET(in_channels=3).to(DEVICE)
    steganolysis = ViT((3, 256, 256), n_patches=16, n_blocks=2, hidden_d=8, n_heads=2, out_d=1)
    embed = StegoEmbeddingLayer()

    # optimizers
    opt_disc = optim.Adam(
        list(disc_X.parameters()) + list(disc_Y.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    opt_gen = optim.Adam(
        list(gen_Y.parameters()) + list(gen_X.parameters()),
        lr=LEARNING_RATE,
        betas=(0.5, 0.999),
    )

    L1 = nn.L1Loss()
    mse = nn.MSELoss()

    if LOAD_MODEL:
        load_checkpoint(
            CHECKPOINT_GEN_X,
            gen_X,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_GEN_Y,
            gen_Y,
            opt_gen,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_DISC_X,
            disc_X,
            opt_disc,
            LEARNING_RATE,
        )
        load_checkpoint(
            CHECKPOINT_DISC_Y,
            disc_Y,
            opt_disc,
            LEARNING_RATE,
        )

    dataset = HorseZebraDataset(
        root_x=TRAIN_DIR + "/horses",
        root_y=TRAIN_DIR + "/zebras",
        transform=transforms_alb,
    )
    val_dataset = HorseZebraDataset(
        root_x=VAL_DIR + "/horses",
        root_y=VAL_DIR + "/zebras",
        transform=transforms_alb,
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=2,
        shuffle=False,
        pin_memory=True,
    )
    loader = DataLoader(
        dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True,
    )
    g_scaler = torch.cuda.amp.GradScaler(enabled=False)
    d_scaler = torch.cuda.amp.GradScaler(enabled=False)


    for epoch in range(NUM_EPOCHS):
        g, d = train_fn(disc_X, disc_Y, gen_Y, gen_X, loader, opt_disc, opt_gen, L1, mse, d_scaler, g_scaler, embed, steganolysis)

        # Evaluate
        scores = pd.DataFrame(columns=['FrechetInceptionDistance', 'InceptionScore_mean', 'InceptionScore_std', 'MSELoss', 'PeakSignalNoiseRatio'])
        for idx, (Y, X) in enumerate(val_loader):
            Y = Y.to(DEVICE).type(torch.float16)
            X = X.to(DEVICE).type(torch.float16)

            # Train Discriminators X and Y
            with torch.cuda.amp.autocast():
                Y_fake = gen_Y(X)
                sc = evaluate(Y, Y_fake)
                scores.loc[idx] = sc

        scores = scores.mean().to_dict()
        scores['Discriminator_loss'] = g
        scores['Generator_loss'] = d
        metrics.loc[epoch] = scores

        if SAVE_MODEL and (epoch%1 == 0):
            # save_checkpoint(gen_X, opt_gen, filename=CHECKPOINT_GEN_X)
            # save_checkpoint(gen_Y, opt_gen, filename=CHECKPOINT_GEN_Y)
            # save_checkpoint(disc_X, opt_disc,
            #                 filename=CHECKPOINT_DISC_X)
            # save_checkpoint(disc_Y, opt_disc,
            #                 filename=CHECKPOINT_DISC_Y)


            torch.save(gen_X.state_dict(), SAVE_PATH+'/GenX.pt')
            torch.save(gen_Y.state_dict(), SAVE_PATH+'/GenY.pt')
            torch.save(disc_X.state_dict(), SAVE_PATH+'/DiscX.pt')
            torch.save(disc_Y.state_dict(), SAVE_PATH+'/DiscY.pt')



In [ ]:
import warnings
warnings.filterwarnings(action='ignore')
metrics = pd.DataFrame(columns=['Discriminator_loss', 'Generator_loss','FrechetInceptionDistance', 'InceptionScore_mean', 'InceptionScore_std', 'MSELoss', 'PeakSignalNoiseRatio'])
seed_everything()
train()

  1%|          | 2/167 [05:27<7:25:59, 162.18s/it, Discriminator_Loss=2.28e+3, Generator_Loss=2.51e+3, X_fake=0.0556, X_real=24.1]

In [ ]:
metrics

,Discriminator_loss,Generator_loss,FrechetInceptionDistance,InceptionScore_mean,InceptionScore_std,MSELoss,PeakSignalNoiseRatio
0,2423.458984,2193.364502,38.435779,1.0,3.041095e-08,16990.459508,5.90778


# SAVE MODELS

In [ ]:
metrics.to_csv("/content/drive/MyDrive/Colab Notebooks/Steganography/ViTMetrics.csv", index=False)